In [1]:
import tensorflow as tf
import experiment_utils as eu
import sys
sys.path.append('../')
from data_processing.stl10 import get_stl10
import importlib
import config as conf

In [2]:
importlib.reload(eu)
config = conf.get_byol()
config.batch_size = 5
ds, _ = get_stl10(batch_size=config.batch_size, split='train')
ds_small = ds.take(5)
experiment = eu.Experiment(config)
experiment.train(ds_small, 'saved_model', epochs=1)


 20%|██        | 1/5 [00:04<00:18,  4.74s/it]

Loss: 20.33536720275879


 40%|████      | 2/5 [00:07<00:11,  3.73s/it]

Loss: 9.67883586883545


 60%|██████    | 3/5 [00:11<00:07,  3.60s/it]

...

Loss: 1.2514212131500244


100%|██████████| 5/5 [00:18<00:00,  3.72s/it]


Epoch 000: Loss: 6.549


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [3]:
experiment.online_network.model.layers[4].symmetry()

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:
import numpy as np
test = np.array(experiment.test[0])
print(test.shape)
test

AttributeError: 'Experiment' object has no attribute 'test'

In [ ]:
import tensorflow as tf
tfm = tf.math
test = tfm.l2_normalize(test, axis=-1)
tfm.reduce_sum(test, axis=-1)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.6404567 , -0.45835656, -0.47579864, -0.5814763 , -0.68898994],
      dtype=float32)>

In [ ]:
import tensorflow_datasets as tfds
ds, ds_info = tfds.load(
    'stl10', split='unlabelled', with_info=True, as_supervised=True)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label


def resize_to_resnet_input(image, label):
    """Resizes images to resnet compatible size (224x224)."""
    return tf.image.resize(image, [224, 224]), label

ds = ds.map(normalize_img,  num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.map(resize_to_resnet_input,  num_parallel_calls=tf.data.AUTOTUNE)
# TODO: for now that's only for pretraining.
# ds = ds.map(lambda img, _: img,  num_parallel_calls=tf.data.AUTOTUNE)

# ds = ds.cache()
# For true randomness, set the shuffle buffer to the full dataset size.
ds = ds.shuffle(1000)
ds = ds.batch(128)

ds = ds.prefetch(tf.data.AUTOTUNE)

for a, b in ds.take(10):
    print(a)

tf.Tensor(
[[[[0.16078432 0.2784314  0.5058824 ]
   [0.16190477 0.2784314  0.50756305]
   [0.16526611 0.2784314  0.5126051 ]
   ...
   [0.22072832 0.34733894 0.5501401 ]
   [0.22577035 0.34565824 0.55518216]
   [0.22745098 0.34509805 0.5568628 ]]

  [[0.16134454 0.27787116 0.5086835 ]
   [0.16230492 0.27795118 0.5096439 ]
   [0.16518608 0.2781913  0.512525  ]
   ...
   [0.2219288  0.34757903 0.5519808 ]
   [0.22649063 0.34613845 0.5560625 ]
   [0.2280112  0.34565827 0.557423  ]]

  [[0.16302522 0.2761905  0.51708686]
   [0.1635054  0.27651063 0.51588637]
   [0.16494599 0.277471   0.51228493]
   ...
   [0.22553024 0.34829932 0.55750304]
   [0.2286515  0.34757903 0.55870354]
   [0.22969188 0.34733894 0.55910367]]

  ...

  [[0.3154061  0.3507001  0.292997  ]
   [0.29147652 0.31476575 0.26538596]
   [0.21968776 0.20696265 0.18255284]
   ...
   [0.16182502 0.1238098  0.12372968]
   [0.23457438 0.18599492 0.16454615]
   [0.2588233  0.20672254 0.17815116]]

  [[0.30532205 0.31708652 0.252660